In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optim

import pandas as pd 
import numpy as np 
import os

In [2]:
PATH = r'/Users/suyashsachdeva/Desktop/GyanBhandar/tox21_dense_train.csv'
data = pd.read_csv(PATH).values
labels = pd.read_csv(PATH).columns
labels

Index(['Unnamed: 0', 'AW', 'AWeight', 'Arto', 'BertzCT', 'Chi0', 'Chi1',
       'Chi10', 'Chi2', 'Chi3',
       ...
       'W3D', 'W3DH', 'WNSA1', 'WNSA2', 'WNSA3', 'WPSA1', 'WPSA2', 'WPSA3',
       'grav', 'rygr'],
      dtype='object', length=802)

In [3]:
for x in range(799):
    if max(data[:, 3:][:, x])!=min(data[:, 3:][:, x]):
        xtr = (data[:, 3:][:, x]-np.mean(data[:, 3:][:, x]))/(max(data[:, 3:][:, x])-min(data[:, 3:][:, x]))
        data[:, 3:][:, x] = xtr
    else:
        xtr = (data[:, 3:][:, x])/(max(data[:, 3:][:, x])+min(data[:, 3:][:, x])+1e-10)+0.5
        data[:, 3:][:, x] = xtr

In [4]:
data = data[:, 3:]

In [5]:
xtrain = np.array(data[:10000], dtype="float32")
xvalid = np.array(data[10000:12000], dtype="float32")

In [16]:
def DenseBlock(x, ndim):
    x = layers.Dense(ndim)(x)
    x = layers.BatchNormalization(momentum=0.5)(x)
    x = layers.LeakyReLU(0.2)(x)
    return  x

def autoencoder(inp):
    inp = layers.Input(inp)
    x = DenseBlock(inp, 256)
    enc_out = DenseBlock(x, 100)
    x = DenseBlock(enc_out, 256)
    dec_out = DenseBlock(x, 799)
    encoder = models.Model(inputs=inp, outputs=enc_out, name="Encoder")
    model = models.Model(inputs=inp, outputs=dec_out, name="Autoencoder")

    return encoder, model

encoder, model = autoencoder((799))
print(encoder.summary())
print(model.summary())

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 799)]             0         
                                                                 
 dense_12 (Dense)            (None, 256)               204800    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 100)               25700     
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                           

In [17]:
model.compile(loss='mse', optimizer="adam", metrics="accuracy")
model.fit(xtrain, xtrain, batch_size=100, verbose=1, epochs=100, validation_data=[xvalid, xvalid], validation_batch_size=100)

Epoch 1/100
100/100 [==============================] - 1s 7ms/step - loss: 0.1145 - accuracy: 0.1294 - val_loss: 0.0398 - val_accuracy: 0.4895
Epoch 2/100
100/100 [==============================] - 1s 6ms/step - loss: 0.0498 - accuracy: 0.4388 - val_loss: 0.0185 - val_accuracy: 0.4855
Epoch 3/100
100/100 [==============================] - 1s 7ms/step - loss: 0.0410 - accuracy: 0.4956 - val_loss: 0.0259 - val_accuracy: 0.5865
Epoch 4/100
100/100 [==============================] - 1s 7ms/step - loss: 0.0356 - accuracy: 0.3344 - val_loss: 0.0332 - val_accuracy: 0.4085
Epoch 5/100
100/100 [==============================] - 1s 6ms/step - loss: 0.0314 - accuracy: 0.2023 - val_loss: 0.0457 - val_accuracy: 0.3095
Epoch 6/100
100/100 [==============================] - 1s 6ms/step - loss: 0.0276 - accuracy: 0.2902 - val_loss: 0.0106 - val_accuracy: 0.4410
Epoch 7/100
100/100 [==============================] - 1s 6ms/step - loss: 0.0225 - accuracy: 0.3019 - val_loss: 0.0167 - val_accuracy: 0.2835

In [18]:
model.save("./model.h5")
encoder.save("./encoder.h5")